In [1]:
from IPython.display import HTML
import tqdm
import gymnasium
import highway_env
from matplotlib import pyplot as plt

env = gymnasium.make('highway-fast-v0', render_mode='rgb_array')
env.reset()

import matplotlib.animation as animation

steps = 5
observations = []

frames = []
for _ in tqdm.trange(steps):
    action = env.unwrapped.action_type.actions_indexes["IDLE"]
    obs, reward, done, truncated, info = env.step(action)
    observations.append(obs)
    frames.append(env.render())

100%|██████████| 5/5 [00:00<00:00, 21.43it/s]


In [2]:
fig, ax = plt.subplots()
ani = animation.ArtistAnimation(fig, [[ax.imshow(frame)] for frame in frames], interval=200, blit=True, repeat_delay=1000)
plt.close(fig)  # Prevent the static image from displaying
HTML(ani.to_jshtml())

In [5]:
import gymnasium as gym
import numpy as np

from highway_env.utils import lmap
from highway_env.road.lane import AbstractLane
from highway_env.vehicle.kinematics import Vehicle

original_obs = np.array(observations)

def rescale(env: gym.Env, obs: np.ndarray) -> np.ndarray:
    feat_names = env.observation_type.features
    side_lanes =env.unwrapped.road.network.all_side_lanes(env.observation_type.observer_vehicle.lane_index)
    features_range = {
                    "x": [-5.0 * Vehicle.MAX_SPEED, 5.0 * Vehicle.MAX_SPEED],
                    "y": [
                        -AbstractLane.DEFAULT_WIDTH * len(side_lanes),
                        AbstractLane.DEFAULT_WIDTH * len(side_lanes),
                    ],
                    "vx": [-2 * Vehicle.MAX_SPEED, 2 * Vehicle.MAX_SPEED],
                    "vy": [-2 * Vehicle.MAX_SPEED, 2 * Vehicle.MAX_SPEED],
                }
    
    rescaled_obs = np.zeros_like(obs)
    for dim_i, feat_name in enumerate(feat_names):
        if feat_name in features_range: 
            x_range = features_range[feat_name]
            rescaled_obs[..., dim_i] = lmap(obs[..., dim_i], [-1, 1], x_range)
        else:
            rescaled_obs[..., dim_i] = obs[..., dim_i]
    return rescaled_obs

rescaled_obs = rescale(env, original_obs)



/u/shuhan/anaconda3/envs/vla_hw/lib/python3.12/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.observation_type to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.observation_type` for environment variables or `env.get_wrapper_attr('observation_type')` that will search the reminding wrappers.
  logger.warn(


In [4]:
import highway_env
import gymnasium as gym
from gymnasium.wrappers import RecordVideo

# Wrap the env by a RecordVideo wrapper
env = gymnasium.make('highway-fast-v0', render_mode='rgb_array')
env = RecordVideo(env, video_folder="run_rescaled",
              episode_trigger=lambda e: True,
              )  # record all episodes

# Provide the video recorder to the wrapped environment
# so it can send it intermediate simulation frames.
env.unwrapped.set_record_video_wrapper(env)

# Record a video as usual
obs, info = env.reset()
done = truncated = False
while not (done or truncated):
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    env.render()
env.close()

# frames = []
# for _ in tqdm.trange(steps):
#     action = env.action_space.sample()
#     obs, reward, done, truncated, info = env.step(action)
#     observations.append(obs)
#     frames.append(env.render())


/u/shuhan/anaconda3/envs/vla_hw/lib/python3.12/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /u/shuhan/projects/vla/notebooks/highway_env/run_rescaled folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
error: XDG_RUNTIME_DIR not set in the environment.


(150, 600, 3)
21
Moviepy - Building video /u/shuhan/projects/vla/notebooks/highway_env/run_rescaled/rl-video-episode-0.mp4.
Moviepy - Writing video /u/shuhan/projects/vla/notebooks/highway_env/run_rescaled/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /u/shuhan/projects/vla/notebooks/highway_env/run_rescaled/rl-video-episode-0.mp4


In [16]:
# import highway_env
# import gymnasium as gym
# from gymnasium.wrappers import RecordVideo
# from stable_baselines3 import DQN

# def visualize_model(model, env, video_folder):
#     # Create and wrap the environment
    
#     env = RecordVideo(env, video_folder=video_folder,
#                       episode_trigger=lambda e: True)  # record all episodes
    
#     # Provide the video recorder to the wrapped environment
#     env.unwrapped.set_record_video_wrapper(env)
    
#     # Record a video
#     obs, info = env.reset()
#     done = truncated = False
#     while not (done or truncated):
#         action, _ = model.predict(obs, deterministic=True)
#         obs, _, done, truncated, _ = env.step(action)
#         env.render()
    
#     env.close()
import sys
sys.path.append('/u/shuhan/projects/vla')
from src.environments.highway_env.visualization import visualize_model

env = gymnasium.make('highway-fast-v0', render_mode='rgb_array')
model_path = "/u/shuhan/projects/vla/data/highway_env/highway_fast_v0_dqn_meta_action/model"
model = DQN.load(model_path)
visualize_model(model, env, "run_RL_model")

/u/shuhan/anaconda3/envs/vla_hw/lib/python3.12/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /u/shuhan/projects/vla/notebooks/highway_env/run_RL_model folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
error: XDG_RUNTIME_DIR not set in the environment.


(150, 600, 3)
151
Moviepy - Building video /u/shuhan/projects/vla/notebooks/highway_env/run_RL_model/rl-video-episode-0.mp4.
Moviepy - Writing video /u/shuhan/projects/vla/notebooks/highway_env/run_RL_model/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /u/shuhan/projects/vla/notebooks/highway_env/run_RL_model/rl-video-episode-0.mp4


In [12]:
env.metadata['render_fps']


10